In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

!mkdir "/kaggle/working/labels"
!cp -r "/kaggle/input/eye-dataset/Fam4a/Fam4a" "/kaggle/working/images"

# Dataset

In [ ]:
imgs = os.listdir('/kaggle/working/images')
imgs_train, imgs_val = train_test_split(imgs, test_size=0.1)

df = pd.read_csv('/kaggle/input/eye-dataset/eye_annotation.csv')
df

In [ ]:
df = df.drop("class", axis=1)
df

In [ ]:
%%time
import yaml
cwd = '/kaggle/working/'
data = dict(
    train =  cwd + 'train.txt',
    val   =  cwd + 'val.txt',
    nc    = 1,
    names = ['eyes'],
)

with open(cwd + 'bgr.yaml', 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)
with open(cwd + 'train.txt', 'w') as f:
    for path in imgs_train:
        f.write(cwd+'images/'+path+'\n')
with open(cwd + 'val.txt', 'w') as f:
    for path in imgs_val:
        f.write(cwd+'images/'+path+'\n')

In [ ]:
for file in imgs:
    file = file.split('.')[0]
    bboxs = []
#     print(file)
    for _,row in df[df['file'] == file].iterrows():
        bbox = [str(0), str(row['Xcent']), str(row['Ycent']), str(row['boxW']), str(row['boxH'])]
        bbox = ' '.join(bbox)
        bboxs.append(bbox)
    with open(cwd+'labels/'+file+'.txt', 'w') as f:
        bboxs = '\n'.join(bboxs)
        f.write(bboxs)


In [ ]:
bboxs

In [ ]:
# %cd /kaggle/working

In [ ]:
# os.listdir("labels")

# YOLOv5

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt

from yolov5 import utils
display = utils.notebook_init()

# Train

In [ ]:
%%time
!python train.py --img 1280\
--batch 8\
--epochs 50\
--data /kaggle/working/bgr.yaml\
--weights yolov5s.pt

# Visualize

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(30,15))
plt.axis('off')
plt.imshow(plt.imread('/kaggle/working/yolov5/runs/train/exp/results.png'))


In [ ]:
fig, ax = plt.subplots(2, 2, figsize = (30,30), constrained_layout = True)
for row in range(1):
    ax[row][0].imshow(plt.imread(f'/kaggle/working/yolov5/runs/train/exp/val_batch{row}_labels.jpg'))
    ax[row][0].set_xticks([])
    ax[row][0].set_yticks([])
    ax[row][0].set_title(f'/kaggle/working/yolov5/runs/train/exp/val_batch{row}_labels.jpg', fontsize = 12)
    
    ax[row][1].imshow(plt.imread(f'/kaggle/working/yolov5/runs/train/exp/val_batch{row}_pred.jpg'))
    ax[row][1].set_xticks([])
    ax[row][1].set_yticks([])
    ax[row][1].set_title(f'/kaggle/working/yolov5/runs/train/exp/val_batch{row}_pred.jpg', fontsize = 12)
plt.show()

# Predict

In [ ]:
import torch
from PIL import Image

CKPT_PATH = '/kaggle/working/yolov5/runs/train/exp/weights/best.pt'
yolov5 = torch.hub.load('/kaggle/working/yolov5','custom',path=CKPT_PATH,source='local',force_reload=True)

'confidence: ' + str(yolov5.conf)

In [ ]:
%cd /kaggle/working

In [ ]:
from PIL import Image
img_test = Image.open("/kaggle/input/my-test-img/photo_2022-11-22_19-21-29.jpg")
img_test

In [ ]:
results = yolov5("/kaggle/input/my-test-img/photo_2022-11-22_19-21-29.jpg")
fig, ax = plt.subplots(figsize=(30, 30))
ax.imshow(results.render()[0])
plt.show()

In [ ]:
import cv2
predict_image = yolov5(["/kaggle/input/my-test-img/photo_2022-11-22_19-21-29.jpg"])
predict_image = predict_image.xyxy[0].data.cpu().numpy().tolist()
predict_image